In [1]:
base_path = "/content/drive/My Drive/Github/mtc-device-activation"

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [3]:
# Load saved data
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append(f'{base_path}/frameworks')

from casbl import casbl

data = np.load('/content/drive/My Drive/Github/mtc-device-activation/data/activity-models/e2-general-model.npz', allow_pickle=True)
all_DeviceLocations = data['all_DeviceLocations']
num_samples = data['num_samples']
device_count = data['device_count']
sparsity = data['sparsity']

data = np.load("/content/drive/My Drive/Github/mtc-device-activation/data/communication-models/e2-communication-simulation-theta.npz", allow_pickle=True)
all_theta = data["all_theta"]

data = np.load("/content/drive/My Drive/Github/mtc-device-activation/data/communication-models/e2-communication-simulation-gamma.npz", allow_pickle=True)
all_gamma = data["all_gamma"]

data = np.load("/content/drive/My Drive/Github/mtc-device-activation/data/communication-models/e2-communication-simulation-z.npz", allow_pickle=True)
all_z = data["all_z"]

data = np.load("/content/drive/My Drive/Github/mtc-device-activation/data/communication-models/e2-communication-simulation-noise.npz", allow_pickle=True)
all_noise = data["all_noise"]

data = np.load("/content/drive/My Drive/Github/mtc-device-activation/data/communication-models/e2-communication-simulation-received-signal.npz", allow_pickle=True)
all_received_signal = data["all_received_signal"]

data = np.load("/content/drive/My Drive/Github/mtc-device-activation/data/communication-models/e2-communication-simulation-general.npz", allow_pickle=True)
snr_db_array = data["snr_db_array"]
pilot_length_array = data["pilot_length_array"]
M = data["M"]

snr_db_index = 2
snr_db=snr_db_array[snr_db_index]

pilot_length_index = 2
pilot_length=pilot_length_array[pilot_length_index]

max_iter = 500

Mounted at /content/drive


In [4]:
sample_range = range(0, 1000)
num_samples = len(sample_range)

In [5]:
# Placeholder arrays for results
nmse_iter_all = np.zeros((num_samples, max_iter))

In [6]:
# Loop through each sample with an outer progress bar
for sample_index in tqdm(sample_range, desc="Processing Samples", position=0):
    gamma = all_gamma[sample_index]  # Ground truth gamma values (true support)
    loc = all_DeviceLocations[sample_index]
    theta = all_theta[sample_index, pilot_length_index, :pilot_length]
    received_signal = all_received_signal[sample_index, pilot_length_index, snr_db_index, :pilot_length]
    z = all_z[sample_index]

    # Compute noise
    snr = 10 ** (snr_db / 10)
    signal_power = np.mean(np.abs(np.matmul(theta, z))**2)
    noise_power = signal_power / snr

    # Run CoSaMP
    gamma_result, z_result, gamma_history, z_history, iteration_count = casbl(
        theta, received_signal, noise_power, loc, max_iter=max_iter
    )

    # NMSE per iteration
    for t, z_est in enumerate(z_history):
        numerator = np.linalg.norm(z[gamma == 1] - z_est[gamma == 1])**2
        denominator = np.linalg.norm(z[gamma == 1])**2
        nmse_iter_all[sample_index, t] = numerator / denominator if denominator != 0 else np.nan

Processing Samples:   3%|▎         | 32/1000 [10:00<5:09:33, 19.19s/it]

Converged after 486 iterations


Processing Samples:   4%|▎         | 37/1000 [11:31<4:53:11, 18.27s/it]

Converged after 448 iterations


Processing Samples:   4%|▍         | 44/1000 [13:38<4:48:50, 18.13s/it]

...

Processing Samples:  96%|█████████▌| 959/1000 [4:53:40<12:14, 17.92s/it]

Converged after 478 iterations


Processing Samples:  98%|█████████▊| 985/1000 [5:01:40<04:31, 18.08s/it]

Converged after 491 iterations


Processing Samples: 100%|██████████| 1000/1000 [5:06:19<00:00, 18.38s/it]


In [7]:
np.savez_compressed(
    f'{base_path}/convergence/data/casbl_snr_db_12_pilot_length_30_convergence.npz',
    nmse_iter_all=nmse_iter_all,
)

print("All results have been saved")

All results have been saved


In [8]:
import numpy as np
import pandas as pd

# Assuming z_result is a complex array of shape (N, M)
N, M = z_result.shape

# Build formatted display table
rows = []
for i in range(N):
    row = {"Device": i}
    for j in range(M):
        row[f"ant{j+1}"] = f"{z_result[i, j].real:.2f}+{z_result[i, j].imag:.2f}j"
    rows.append(row)

df = pd.DataFrame(rows)

# Set column display options
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

# Print as a clean string
print(df.to_string(index=False))

 Device         ant1         ant2         ant3         ant4
      0   0.00+0.00j  0.00+-0.00j  -0.00+0.00j -0.00+-0.00j
      1 -0.05+-0.00j  -0.45+0.21j -0.38+-0.08j  0.02+-0.36j
      2   0.70+1.68j   0.12+0.79j   0.04+0.01j   0.38+0.40j
      3  -0.00+0.00j  -0.00+0.00j -0.00+-0.00j -0.00+-0.00j
      4 -0.00+-0.00j  0.00+-0.00j   0.00+0.00j  0.00+-0.00j
      5 -0.00+-0.00j   0.00+0.00j  -0.00+0.00j  0.00+-0.00j
      6   0.19+0.16j -0.20+-0.04j   0.34+0.06j  0.16+-0.32j
      7  -0.00+0.00j  -0.00+0.00j  0.00+-0.00j -0.00+-0.00j
      8   0.00+0.00j -0.00+-0.00j  -0.00+0.00j  -0.00+0.00j
      9  0.00+-0.00j  0.00+-0.00j  0.00+-0.00j   0.00+0.00j
     10   0.00+0.00j -0.00+-0.00j   0.00+0.00j   0.00+0.00j
     11  -0.00+0.00j   0.00+0.00j  -0.00+0.00j -0.00+-0.00j
     12 -0.00+-0.00j   0.00+0.00j   0.00+0.00j -0.00+-0.00j
     13  -0.00+0.00j  0.00+-0.00j   0.00+0.01j  0.00+-0.00j
     14  -0.00+0.00j   0.00+0.00j -0.00+-0.00j -0.00+-0.00j
     15 -0.00+-0.00j   0.00+0.00j  -0.00